## assess the relevance

In [109]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

import pandas as pd

from textblob import TextBlob
import numpy as np

import re
import nltk
from nltk.corpus import stopwords

## BBC

In [124]:
df10= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2010_2.csv')
df11= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2011_2.csv')
df12= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2012_2.csv')
df13= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2013_2.csv')
df14= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2014_2.csv')
df15= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2015_2.csv')
df16= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2016_2.csv')
df17= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2017_2.csv')
df18= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2018_2.csv')
df19= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2019_2.csv')
df20= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2020_2.csv')
df21= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2021_2.csv')
df22= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2022_2.csv')
df23= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2023_2.csv')



In [104]:
df23

,title,date,url,player,textblob_sentiment
0,NBA All-Star Game: Team Giannis beats Team LeB...,2023,https://www.bbc.co.uk/sport/basketball/64701884,Jayson Tatum,0.076734
1,NBA round-up: Boston Celtics extend winning ru...,2023,https://www.bbc.co.uk/sport/basketball/63614776,Jayson Tatum,0.220617
2,NBA: Kevin Durant scores 23 points on Phoenix ...,2023,https://www.bbc.co.uk/sport/basketball/64820742,Jayson Tatum,0.163115
3,NBA: Dallas Mavericks stars Luka Doncic and Ky...,2023,https://www.bbc.co.uk/sport/basketball/64833608,Jayson Tatum,0.141611
4,NBA on the BBC: The big questions as NBA retur...,2023,https://www.bbc.co.uk/sport/basketball/64722489,Jayson Tatum,0.063053
...,...,...,...,...,...
949,Luka Doncic scores magical 60-point triple-dou...,2023,https://www.bbc.co.uk/sport/basketball/64107654,Kevin Durant,0.205116
950,NBA Slam Dunk Contest: Mac McClung signs for P...,2023,https://www.bbc.co.uk/sport/basketball/64695886,Kevin Durant,0.195479
951,NBA: Defending champions Golden State Warriors...,2023,https://www.bbc.co.uk/sport/basketball/63933734,Kevin Durant,0.098578
952,NBA: Zion Williamson returns as New Orleans Pe...,2023,https://www.bbc.co.uk/sport/basketball/63325545,Kevin Durant,0.068242


In [107]:
def get_bbc_content(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser") 
    content = soup.find("div", attrs={"class": "gel-layout__item gel-2/3@l"})
    
    if content is None:
        content = soup.find("td", attrs={"class": "storybody"})
        if content is None:
            return None

    main_body = content.find_all('p')
    text = ""
    for i in main_body:
        text += i.text
    clean_text = text.replace('\n', ' ').replace('\xa0', '')
    return clean_text


def preprocess_text(text):
    text = text.lower()     # Convert text to lowercase
    
    text = re.sub(r"[^\w\s]", "", text) # Remove punctuation and special characters
    
    words = nltk.word_tokenize(text)     # Tokenize the text into words

    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    
    preprocessed_text = " ".join(words)
    
    return preprocessed_text







In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics.pairwise import cosine_similarity

def assess_bbc_relevance(df):
    relevance_scores = []

    for index, row in df.iterrows():
        content = get_bbc_content(row['url'])
        player_name = row['player']

        if content is None:
            content = row['title']

        statement = player_name + " will be the MVP"

        # Preprocess content and statement
        preprocessed_content = preprocess_text(content)
        preprocessed_statement = preprocess_text(statement)

        # Calculate TF-IDF scores
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([preprocessed_content, preprocessed_statement])

        # Calculate cosine similarity between content and statement
        similarity_score = cosine_similarity(tfidf_matrix)[0, 1]

        # Normalize similarity score to range between 0 and 1
        normalized_score = similarity_score / tfidf_matrix[0].sum()

        relevance_scores.append(normalized_score)

    # Normalize relevance scores to range between 0 and 1
    scaler = MaxAbsScaler()
    normalized_scores = scaler.fit_transform(np.array(relevance_scores).reshape(-1, 1))
    df['relevance_score'] = normalized_scores

    return df


In [119]:
df_10 = assess_bbc_relevance(df10)
df_10.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2010_3.csv', index=False)


In [120]:
df_11 = assess_bbc_relevance(df11)
df_11.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2011_3.csv', index=False)


In [125]:
df_12 = assess_bbc_relevance(df12)
df_12.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2012_3.csv', index=False)


In [126]:
df_13 = assess_bbc_relevance(df13)
df_13.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2013_3.csv', index=False)


In [ ]:
df_14 = assess_bbc_relevance(df14)
df_14.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2014_3.csv', index=False)


In [127]:
df_15 = assess_bbc_relevance(df15)
df_15.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2015_3.csv', index=False)


In [128]:
df_16 = assess_bbc_relevance(df16)
df_16.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2016_3.csv', index=False)


In [129]:
df_17 = assess_bbc_relevance(df17)
df_17.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2017_3.csv', index=False)


In [130]:
df_18 = assess_bbc_relevance(df18)
df_18.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2018_3.csv', index=False)


In [131]:
df_19 = assess_bbc_relevance(df19)
df_19.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2019_3.csv', index=False)


In [132]:
df_20 = assess_bbc_relevance(df20)
df_20.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2020_3.csv', index=False)


In [133]:
df_21 = assess_bbc_relevance(df20)
df_21.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2021_3.csv', index=False)


In [134]:
df_22 = assess_bbc_relevance(df22)
df_22.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2022_3.csv', index=False)


In [135]:
df_23 = assess_bbc_relevance(df23)
df_23.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/bbc_seperate/bbc_2023_3.csv', index=False)


In [142]:
df14.to_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/news_data/bbc_seperate/bbc_2014_3.csv', index=False)
